In [1]:
!pip install wikipedia-api

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install rake-nltk

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install rake-nltk


Defaulting to user installation because normal site-packages is not writeable


In [4]:

!pip install spacy


Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install keybert

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install textblob

Defaulting to user installation because normal site-packages is not writeable


In [7]:
!pip install yake

Defaulting to user installation because normal site-packages is not writeable


In [8]:

!python -m spacy download en_core_web_sm



Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB 

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\bhumi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\bhumi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\bhumi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\bhumi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-pa

In [9]:
!pip install matplotlib


Defaulting to user installation because normal site-packages is not writeable


In [10]:
!pip install textdistance


Defaulting to user installation because normal site-packages is not writeable


In [95]:
import wikipedia
from keybert import KeyBERT
from rake_nltk import Rake
import spacy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
from collections import Counter

# Uncomment this line if you haven't downloaded the NLTK stopwords
# nltk.download('stopwords')

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize KeyBERT model
kw_model = KeyBERT(model="distilbert-base-nli-mean-tokens")

# Function to clean and preprocess text
def preprocess_text(text):
    text = re.sub(r"\[.*?\]", "", text)  # Remove references
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text)  # Remove extra whitespaces
    return text.strip()

# Function to extract keywords using KeyBERT
def extract_keybert_keywords(text):
    try:
        word_count = len(text.split())
        top_n = max(1, word_count // 10)
        keywords = kw_model.extract_keywords(
            docs=text,
            keyphrase_ngram_range=(1, 3),
            top_n=top_n,
            use_maxsum=False,
        )
        return [kw[0] for kw in keywords]
    except Exception as e:
        print(f"KeyBERT error: {e}")
        return []

# Function to extract keywords using RAKE
def extract_rake_keywords(text):
    rake = Rake()
    rake.extract_keywords_from_text(text)
    return rake.get_ranked_phrases()[:10]

# Function to extract keywords using SpaCy
def extract_spacy_keywords(text):
    doc = nlp(text)
    return [chunk.text for chunk in doc.noun_chunks][:10]

# Function to refine keywords
def refine_keywords(content):
    preprocessed_text = preprocess_text(content)

    print("\nExtracting keywords using KeyBERT...")
    keybert_keywords = extract_keybert_keywords(preprocessed_text)

    print("Extracting keywords using RAKE...")
    rake_keywords = extract_rake_keywords(preprocessed_text)

    print("Extracting keywords using SpaCy...")
    spacy_keywords = extract_spacy_keywords(preprocessed_text)

    # Combine and clean keywords
    all_keywords = keybert_keywords + rake_keywords + spacy_keywords
    unique_keywords = list(set(all_keywords))  # Remove duplicates
    return unique_keywords

# Function to fetch Wikipedia page content
def fetch_wikipedia_page(query, word_limit=1000):
    if query.startswith("http"):
        url = query  # Use the full URL if provided
    else:
        base_url = "https://en.wikipedia.org/wiki/"
        formatted_query = query.replace(" ", "_")
        url = base_url + formatted_query

    print(f"Fetching content from: {url}")
    try:
        response = requests.get(url, headers={"User-Agent": "MyWikipediaBot/1.0"})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        content = []
        for paragraph in soup.find_all("p"):
            text = paragraph.get_text(strip=True)
            if text:
                content.append(text)
        full_text = " ".join(content)
        words = full_text.split()
        return " ".join(words[:word_limit]) if len(words) > word_limit else full_text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return None

# Function to fetch directly connected links from Wikipedia
def fetch_direct_links(query):
    base_url = "https://en.wikipedia.org/wiki/"
    formatted_query = query.replace(" ", "_")
    url = base_url + formatted_query

    print(f"Fetching links from: {url}")
    try:
        response = requests.get(url, headers={"User-Agent": "MyWikipediaBot/1.0"})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        links = []
        link_texts = []
        for paragraph in soup.find_all("p"):  # Focus on paragraphs for meaningful content
            for link in paragraph.find_all("a", href=True):
                href = link['href']
                if href.startswith("/wiki/") and not re.search(r":", href):  # Exclude special pages
                    full_url = urljoin(base_url, href)
                    links.append(full_url)
                    link_texts.append(link.get_text(strip=True))  # Text of the anchor tag
        
        # Remove duplicates by converting to set and back to list
        unique_links = list(set(links))
        unique_link_texts = [link_texts[links.index(link)] for link in unique_links]
        
        return unique_links, unique_link_texts
    except requests.exceptions.RequestException as e:
        print(f"Error fetching links: {e}")
        return [], []
    

#TRY1 - consine/tfidf

# # Function to calculate similarity and filter top links
# def filter_top_links(query, links, link_texts, top_n=10):
#     if not links or not link_texts:
#         print("No links or texts available for filtering.")
#         return []

#     try:
#         vectorizer = TfidfVectorizer()
#         # Combine query with anchor texts into the corpus
#         corpus = [query] + link_texts
#         tfidf_matrix = vectorizer.fit_transform(corpus)

#         # Compute cosine similarity between query (first entry) and anchor texts
#         similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
#         link_similarity = list(zip(links, similarities))

#         # Sort links by similarity in descending order and select top_n
#         sorted_links = sorted(link_similarity, key=lambda x: x[1], reverse=True)[:top_n]
#         return [link for link, _ in sorted_links]
#     except Exception as e:
#         print(f"Error in similarity calculation: {e}")
#         return []


#TRY2- WUPALMER 
# from nltk.corpus import wordnet as wn

# def calculate_wu_palmer_similarity(word1, word2):
#     try:
#         # Get the synsets for the given words
#         synsets1 = wn.synsets(word1)
#         synsets2 = wn.synsets(word2)

#         if not synsets1 or not synsets2:
#             return 0  # Return 0 similarity if no synsets are found

#         # Compute the maximum Wu-Palmer similarity between all synset pairs
#         max_similarity = max(
#             (s1.wup_similarity(s2) or 0) for s1 in synsets1 for s2 in synsets2
#         )
#         return max_similarity
#     except Exception as e:
#         print(f"Error calculating Wu-Palmer similarity: {e}")
#         return 0

# def filter_top_links(query, links, link_texts, top_n=10):
#     if not links or not link_texts:
#         print("No links or texts available for filtering.")
#         return []

#     try:
#         # Compute similarity scores between query and each link text
#         similarities = []
#         for text in link_texts:
#             words_query = query.split()
#             words_text = text.split()
#             # Calculate average Wu-Palmer similarity for all word pairs
#             similarity = sum(
#                 calculate_wu_palmer_similarity(w1, w2)
#                 for w1 in words_query
#                 for w2 in words_text
#             ) / (len(words_query) * len(words_text))
#             similarities.append(similarity)

#         link_similarity = list(zip(links, similarities))

#         # Sort links by similarity in descending order and select top_n
#         sorted_links = sorted(link_similarity, key=lambda x: x[1], reverse=True)[:top_n]
#         return [link for link, _ in sorted_links]
#     except Exception as e:
#         print(f"Error in similarity calculation: {e}")
#         return []

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet as wn

def calculate_wu_palmer_similarity(word1, word2):
    try:
        # Get the synsets for the given words
        synsets1 = wn.synsets(word1)
        synsets2 = wn.synsets(word2)

        if not synsets1 or not synsets2:
            return 0  # Return 0 similarity if no synsets are found

        # Compute the maximum Wu-Palmer similarity between all synset pairs
        max_similarity = max(
            (s1.wup_similarity(s2) or 0) for s1 in synsets1 for s2 in synsets2
        )
        return max_similarity
    except Exception as e:
        print(f"Error calculating Wu-Palmer similarity: {e}")
        return 0

def calculate_combined_similarity(query, text, cosine_weight=0.7, semantic_weight=0.):
    # Calculate cosine similarity
    vectorizer = TfidfVectorizer()
    corpus = [query, text]
    tfidf_matrix = vectorizer.fit_transform(corpus)
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()[0]

    # Calculate semantic similarity
    words_query = query.split()
    words_text = text.split()
    if words_query and words_text:
        semantic_sim = sum(
            calculate_wu_palmer_similarity(w1, w2)
            for w1 in words_query
            for w2 in words_text
        ) / (len(words_query) * len(words_text))
    else:
        semantic_sim = 0

    # Combine the two scores
    combined_score = (cosine_weight * cosine_sim) + (semantic_weight * semantic_sim)
    return combined_score




#filter_top_links function
def filter_top_links(query, links, link_texts, top_n=10):
    if not links or not link_texts:
        print("No links or texts available for filtering.")
        return []

    try:
        # Compute combined similarity scores for each link
        similarities = [
            calculate_combined_similarity(query, text)
            for text in link_texts
        ]
        link_similarity = list(zip(links, link_texts, similarities))  # Include link text in tuple

        # Sort links by combined similarity in descending order and select top_n
        sorted_links = sorted(link_similarity, key=lambda x: x[2], reverse=True)[:top_n]

        # Return the sorted links (top 15)
        return [link for link, _, _ in sorted_links]
    except Exception as e:
        print(f"Error in similarity calculation: {e}")
        return []
    




# Main function to fetch keywords
def fetch_and_extract_keywords(query):
    main_content = fetch_wikipedia_page(query)
    if not main_content:
        print("No content fetched for the main page.")
        return []

    print("\nExtracting keywords from the main page...")
    keywords_list = refine_keywords(main_content)

    outlinks, link_texts = fetch_direct_links(query)
    top_links = filter_top_links(query, outlinks, link_texts)



    for idx, link in enumerate(top_links):
        try:
            print(f"\nProcessing outlink {idx + 1}/{len(top_links)}: {link}")
            linked_content = fetch_wikipedia_page(link.split("/")[-1])
            if linked_content:
                print(f"Extracting keywords from: {link}")
                keywords_list.extend(refine_keywords(linked_content))
        except Exception as e:
            print(f"Error processing outlink {link}: {e}")

    return list(set(keywords_list))  # Return unique keywords

# Entry point
if __name__ == "__main__":
    query_keyword = input("Enter the keyword to search in Wikipedia: ")
    final_keywords = fetch_and_extract_keywords(query_keyword)
    print("\nFinal Keywords:")
    print(final_keywords)

Fetching content from: https://en.wikipedia.org/wiki/ice_cream

Extracting keywords from the main page...

Extracting keywords using KeyBERT...
Extracting keywords using RAKE...
Extracting keywords using SpaCy...
Fetching links from: https://en.wikipedia.org/wiki/ice_cream

Processing outlink 1/10: https://en.wikipedia.org/wiki/Vanilla_ice_cream
Fetching content from: https://en.wikipedia.org/wiki/Vanilla_ice_cream
Extracting keywords from: https://en.wikipedia.org/wiki/Vanilla_ice_cream

Extracting keywords using KeyBERT...
Extracting keywords using RAKE...
Extracting keywords using SpaCy...

Processing outlink 2/10: https://en.wikipedia.org/wiki/Ice_cream_parlour
Fetching content from: https://en.wikipedia.org/wiki/Ice_cream_parlour
Extracting keywords from: https://en.wikipedia.org/wiki/Ice_cream_parlour

Extracting keywords using KeyBERT...
Extracting keywords using RAKE...
Extracting keywords using SpaCy...

Processing outlink 3/10: https://en.wikipedia.org/wiki/Ice_cream_van
Fetc

In [96]:
len(final_keywords)

865

In [97]:
def split_into_keywords(keywords):
    split_keywords = []
    for keyword in keywords:
        words = keyword.split()  # Split the string by space
        split_keywords.extend(words)  # Add all words to the list
    return split_keywords


preprocessed_keywords = split_into_keywords(final_keywords)
print(preprocessed_keywords)


['ice', 'cream', 'cups', 'baking', 'biscuitcups', 'cream', 'vans', 'carried', 'frozen', 'dairy', 'dessert', 'ice', 'cream', 'aroyal', 'cream', 'soda', 'vanilla', 'flavor', 'natural', 'vanilla', 'consumed', 'vanilla', 'drinks', 'honor', 'ofnational', 'ice', 'cream', 'month', 'several', 'prominent', 'publications', 'rank', 'syrup', 'base', 'soda', 'ice', 'cream', 'served', 'It', '1686', 'francesco', 'procopio', 'del', 'coltelli', 'opened', 'paris', 'first', 'café', 'thecafé', 'procope', 'named', 'versionis', 'blended', 'milkshake', 'anice', 'cream', 'coneengland', 'coffee', 'cream', '10', 'cream', 'company', 'chocolate', 'indian', 'ice', 'cream', 'company', 'chocolate', 'shoppe', 'creamsoft', 'serveis', 'softer', 'used', 'flavor', 'chocolate', 'cones', 'cream', 'truckamerican', 'available', 'ice', 'cream', 'used', 'vanilla', 'flavor', 'savorycustards', 'normally', 'milk', 'ice', 'cream', 'merchantsantonio', 'valvona', 'frommanchester', 'patented', 'creamcan', 'carried', 'eaten', 'prepara

In [98]:
len(preprocessed_keywords)

2956

In [99]:
from nltk.corpus import words
import nltk

# Ensure the words corpus is downloaded
nltk.download('words')

word_list = set(words.words())

def split_compound_words(keywords):
    def split_word(word):
        # If the word itself is valid, return it
        if word in word_list:
            return [word]
        
        # Otherwise, try to split it
        for i in range(len(word), 0, -1):  # Start from the longest possible split
            prefix = word[:i]
            suffix = word[i:]
            if prefix in word_list:
                return [prefix] + split_word(suffix)
        
        # If no valid split is found, return the word itself
        return [word]

    split_keywords = []
    for keyword in keywords:
        split_keywords.extend(split_word(keyword))
    return split_keywords

split_keywords = split_compound_words(preprocessed_keywords)
print(split_keywords)
preprocessed_keywords = split_compound_words(preprocessed_keywords)

['ice', 'cream', 'cup', 's', 'baking', 'biscuit', 'cup', 's', 'cream', 'van', 's', 'carried', 'frozen', 'dairy', 'dessert', 'ice', 'cream', 'ar', 'o', 'ya', 'l', 'cream', 'soda', 'vanilla', 'flavor', 'natural', 'vanilla', 'consume', 'd', 'vanilla', 'drink', 's', 'honor', 'of', 'national', 'ice', 'cream', 'month', 'several', 'prominent', 'publication', 's', 'rank', 'syrup', 'base', 'soda', 'ice', 'cream', 'serve', 'd', 'I', 't', '1686', 'franc', 'es', 'c', 'o', 'pro', 'cop', 'io', 'de', 'l', 'colt', 'ell', 'i', 'open', 'e', 'd', 'pari', 's', 'first', 'ca', 'f', 'é', 'theca', 'f', 'é', 'pro', 'cope', 'name', 'd', 'version', 'is', 'blended', 'milk', 'shake', 'ani', 'ce', 'cream', 'coneen', 'gland', 'coffee', 'cream', '10', 'cream', 'company', 'chocolate', 'in', 'dian', 'ice', 'cream', 'company', 'chocolate', 'shoppe', 'cream', 'soft', 'serve', 'is', 'soft', 'er', 'used', 'flavor', 'chocolate', 'cones', 'cream', 'truck', 'ame', 'r', 'i', 'can', 'available', 'ice', 'cream', 'used', 'vanilla

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\bhumi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [100]:
len(preprocessed_keywords)

5008

In [101]:
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')

# Define the stop_words set
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from keywords
def remove_stopwords(keywords):
    return [word for word in keywords if word.lower() not in stop_words]


filtered_keywords = remove_stopwords(preprocessed_keywords)
print(filtered_keywords)


['ice', 'cream', 'cup', 'baking', 'biscuit', 'cup', 'cream', 'van', 'carried', 'frozen', 'dairy', 'dessert', 'ice', 'cream', 'ar', 'ya', 'l', 'cream', 'soda', 'vanilla', 'flavor', 'natural', 'vanilla', 'consume', 'vanilla', 'drink', 'honor', 'national', 'ice', 'cream', 'month', 'several', 'prominent', 'publication', 'rank', 'syrup', 'base', 'soda', 'ice', 'cream', 'serve', '1686', 'franc', 'es', 'c', 'pro', 'cop', 'io', 'de', 'l', 'colt', 'ell', 'open', 'e', 'pari', 'first', 'ca', 'f', 'é', 'theca', 'f', 'é', 'pro', 'cope', 'name', 'version', 'blended', 'milk', 'shake', 'ani', 'ce', 'cream', 'coneen', 'gland', 'coffee', 'cream', '10', 'cream', 'company', 'chocolate', 'dian', 'ice', 'cream', 'company', 'chocolate', 'shoppe', 'cream', 'soft', 'serve', 'soft', 'er', 'used', 'flavor', 'chocolate', 'cones', 'cream', 'truck', 'ame', 'r', 'available', 'ice', 'cream', 'used', 'vanilla', 'flavor', 'savory', 'custard', 'normally', 'milk', 'ice', 'cream', 'merchant', 'santon', 'io', 'v', 'al', 'v

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhumi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [102]:
def filter_short_words(keywords):
    # Filter out words that are 1 or 2 characters long
    return [word for word in keywords if len(word) > 2]


filtered_keywords = filter_short_words(filtered_keywords)
print(filtered_keywords)

['ice', 'cream', 'cup', 'baking', 'biscuit', 'cup', 'cream', 'van', 'carried', 'frozen', 'dairy', 'dessert', 'ice', 'cream', 'cream', 'soda', 'vanilla', 'flavor', 'natural', 'vanilla', 'consume', 'vanilla', 'drink', 'honor', 'national', 'ice', 'cream', 'month', 'several', 'prominent', 'publication', 'rank', 'syrup', 'base', 'soda', 'ice', 'cream', 'serve', '1686', 'franc', 'pro', 'cop', 'colt', 'ell', 'open', 'pari', 'first', 'theca', 'pro', 'cope', 'name', 'version', 'blended', 'milk', 'shake', 'ani', 'cream', 'coneen', 'gland', 'coffee', 'cream', 'cream', 'company', 'chocolate', 'dian', 'ice', 'cream', 'company', 'chocolate', 'shoppe', 'cream', 'soft', 'serve', 'soft', 'used', 'flavor', 'chocolate', 'cones', 'cream', 'truck', 'ame', 'available', 'ice', 'cream', 'used', 'vanilla', 'flavor', 'savory', 'custard', 'normally', 'milk', 'ice', 'cream', 'merchant', 'santon', 'ona', 'manche', 'patent', 'cream', 'carried', 'eaten', 'preparation', 'vanilla', 'ice', 'cream', 'cream', 'cones', 'i

In [103]:

filtered_keywords = [item.lower() for item in filtered_keywords]

In [104]:
def remove_duplicates(keywords):
    return list(set(keywords))

filtered_keywords=remove_duplicates(filtered_keywords)

In [105]:
len(filtered_keywords)

835

In [106]:
print(filtered_keywords)

['sweet', 'ate', 'butter', 'sundae', 'style', 'owner', 'notable', 'gin', 'overnight', 'stein', 'dense', 'sanders', 'pen', 'tost', 'ill', 'characterize', 'cheese', '1790', 'ell', 'air', 'cones', '1922but', 'insulated', 'often', '100vanillin', 'geo', 'preparation', 'shop', 'gel', 'selection', 'equip', 'pretzel', 'adhere', 'butterfat', 'several', 'serving', 'area', 'ers', 'crunch', 'wrapper', 'anice', 'leading', 'hau', 'follow', 'flag', 'topping', 'orf', 'topped', 'people', 'war', 'soured', 'beg', 'coneen', 'prod', 'fruit', 'land', 'salt', 'commercial', 'label', 'addition', 'three', 'cone', 'illicit', 'bean', 'sauce', 'calcium', 'mixed', 'regulation', 'origin', 'thicken', 'cocoa', 'brit', 'juice', 'costa', 'official', 'company', 'queen', 'inness', 'state', 'pink', 'produce', 'sad', 'boy', 'country', 'vanilla', 'sell', 'elaborate', '1682le', 'four', 'sour', '1872', 'thin', 'remain', 'cooking', 'yolk', 'different', '1920s', 'bottle', 'swiss', 'full', 'retail', 'parker', 'create', '3558but',

In [107]:
print(query_keyword)

ice cream


In [108]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# BERT-based similarity
def sentence_transformer_similarity(query, keywords):
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    embeddings = model.encode([query] + keywords)
    return cosine_similarity([embeddings[0]], embeddings[1:]).flatten()

# Calculate and sort similarities
def calculate_and_sort_similarities(query_keyword, filtered_keywords):
    # Compute BERT similarity for all keywords
    bert_scores = sentence_transformer_similarity(query_keyword, filtered_keywords)

    # Store results
    results = {"BERT Similarity (SentenceTransformer)": []}

    for keyword, bert_sim in zip(filtered_keywords, bert_scores):
        results["BERT Similarity (SentenceTransformer)"].append((keyword, bert_sim))

    # Sort results by score
    results["BERT Similarity (SentenceTransformer)"].sort(key=lambda x: x[1], reverse=True)

    # Print the top 10 keywords
    print("\n--- Top Keywords by BERT Similarity ---")
    for keyword, score in results["BERT Similarity (SentenceTransformer)"][:10]:
        print(f"{keyword} (Score: {score:.4f})")

    # Return top 100 keywords by BERT similarity
    return [kw for kw, _ in results["BERT Similarity (SentenceTransformer)"][:100]]



# Compute and display results
top_keywords = calculate_and_sort_similarities(query_keyword, filtered_keywords)
print("\nTop 100 Keywords by BERT Similarity:", top_keywords)



--- Top Keywords by BERT Similarity ---
cream (Score: 0.7949)
ice (Score: 0.6899)
creamery (Score: 0.6866)
dessert (Score: 0.5985)
frozen (Score: 0.5701)
creamy (Score: 0.5466)
vanilla (Score: 0.5329)
frosting (Score: 0.5143)
pastry (Score: 0.4938)
chocolate (Score: 0.4838)

Top 100 Keywords by BERT Similarity: ['cream', 'ice', 'creamery', 'dessert', 'frozen', 'creamy', 'vanilla', 'frosting', 'pastry', 'chocolate', 'pudding', 'oil', 'butter', 'edible', 'flavor', 'flavored', 'chilled', 'flavoring', 'syrup', 'treat', 'baked', 'scoop', 'cake', 'dairy', 'cone', 'milk', 'waffle', 'food', 'coffee', 'candy', 'gel', 'snack', 'whey', 'cocoa', 'cheese', 'caramel', 'butterfat', 'biscuit', 'confection', 'curling', 'sauce', 'extract', 'powder', 'coneen', 'confectioner', 'poke', 'ginger', 'acid', 'isle', 'lick', 'sweeten', 'salt', 'pie', 'sweet', 'cold', 'flavour', 'liquid', 'baking', 'pink', 'emulsion', 'texture', 'humor', 'soft', 'bean', 'snow', 'shake', 'cup', 'ate', 'fat', 'whipped', 'cones', '

In [109]:
print(top_keywords)

['cream', 'ice', 'creamery', 'dessert', 'frozen', 'creamy', 'vanilla', 'frosting', 'pastry', 'chocolate', 'pudding', 'oil', 'butter', 'edible', 'flavor', 'flavored', 'chilled', 'flavoring', 'syrup', 'treat', 'baked', 'scoop', 'cake', 'dairy', 'cone', 'milk', 'waffle', 'food', 'coffee', 'candy', 'gel', 'snack', 'whey', 'cocoa', 'cheese', 'caramel', 'butterfat', 'biscuit', 'confection', 'curling', 'sauce', 'extract', 'powder', 'coneen', 'confectioner', 'poke', 'ginger', 'acid', 'isle', 'lick', 'sweeten', 'salt', 'pie', 'sweet', 'cold', 'flavour', 'liquid', 'baking', 'pink', 'emulsion', 'texture', 'humor', 'soft', 'bean', 'snow', 'shake', 'cup', 'ate', 'fat', 'whipped', 'cones', 'dried', 'sundae', 'eaten', 'porcelain', 'ingredient', 'freezer', 'solid', 'yoghurt', 'cool', 'condensed', 'freeze', 'oat', 'brown', 'beverage', 'juice', 'product', 'skim', 'lip', 'soup', 'recipe', 'spoon', 'fruity', 'canned', 'fish', 'roll', 'sla', 'item', 'chunk', 'container']


In [110]:
import json
import os


# Print the list to verify contents
print("Top Keywords:", top_keywords)

# Define the file name
file_name = "filtered_keywords.json"

# Save the list to a JSON file
with open(file_name, "w", encoding="utf-8") as f:
    json.dump(top_keywords, f, indent=4, ensure_ascii=False)

# Get the absolute path of the file
file_path = os.path.abspath(file_name)

# Print the file path
print(f"Filtered keywords saved to: {file_path}")


Top Keywords: ['cream', 'ice', 'creamery', 'dessert', 'frozen', 'creamy', 'vanilla', 'frosting', 'pastry', 'chocolate', 'pudding', 'oil', 'butter', 'edible', 'flavor', 'flavored', 'chilled', 'flavoring', 'syrup', 'treat', 'baked', 'scoop', 'cake', 'dairy', 'cone', 'milk', 'waffle', 'food', 'coffee', 'candy', 'gel', 'snack', 'whey', 'cocoa', 'cheese', 'caramel', 'butterfat', 'biscuit', 'confection', 'curling', 'sauce', 'extract', 'powder', 'coneen', 'confectioner', 'poke', 'ginger', 'acid', 'isle', 'lick', 'sweeten', 'salt', 'pie', 'sweet', 'cold', 'flavour', 'liquid', 'baking', 'pink', 'emulsion', 'texture', 'humor', 'soft', 'bean', 'snow', 'shake', 'cup', 'ate', 'fat', 'whipped', 'cones', 'dried', 'sundae', 'eaten', 'porcelain', 'ingredient', 'freezer', 'solid', 'yoghurt', 'cool', 'condensed', 'freeze', 'oat', 'brown', 'beverage', 'juice', 'product', 'skim', 'lip', 'soup', 'recipe', 'spoon', 'fruity', 'canned', 'fish', 'roll', 'sla', 'item', 'chunk', 'container']
Filtered keywords sav

In [111]:
# import json
# from itertools import islice
# from nltk.corpus import wordnet as wn
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from nltk.corpus import stopwords
# from nltk.metrics import edit_distance
# from difflib import SequenceMatcher
# from sentence_transformers import SentenceTransformer
# import numpy as np

# # Pre-trained SentenceTransformer model
# model = SentenceTransformer('all-MiniLM-L6-v2')

# # Wu-Palmer similarity
# def wu_palmer_similarity(word1, word2):
#     syn1 = wn.synsets(word1, pos=wn.NOUN)
#     syn2 = wn.synsets(word2, pos=wn.NOUN)
#     if syn1 and syn2:
#         return max((s1.wup_similarity(s2) or 0) for s1 in syn1[:2] for s2 in syn2[:2])
#     return 0

# # Cosine similarity using TF-IDF
# def compute_cosine_similarity(query, keywords):
#     vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=10000)
#     vectors = vectorizer.fit_transform([query] + keywords)
#     return cosine_similarity(vectors[0:1], vectors[1:]).flatten()

# # Semantic similarity using SequenceMatcher
# def semantic_similarity(word1, word2):
#     return SequenceMatcher(None, word1, word2).ratio()

# # Jaccard similarity
# def jaccard_similarity(word1, word2):
#     set1, set2 = set(word1), set(word2)
#     intersection = set1 & set2
#     union = set1 | set2
#     return len(intersection) / len(union) if union else 0

# # Levenshtein similarity
# def levenshtein_similarity(word1, word2):
#     max_len = max(len(word1), len(word2))
#     return 1 - (edit_distance(word1, word2) / max_len) if max_len > 0 else 0

# # Sentence Transformer similarity
# def sentence_transformer_similarity(query, keywords):
#     embeddings = model.encode([query] + keywords)
#     query_embedding = embeddings[0]
#     keyword_embeddings = embeddings[1:]
#     return cosine_similarity([query_embedding], keyword_embeddings).flatten()

# # Batch processing
# def batch_process(iterable, batch_size):
#     it = iter(iterable)
#     while batch := list(islice(it, batch_size)):
#         yield batch

# # Calculate and sort similarities
# def calculate_and_sort_similarities(query_keyword, filtered_keywords):
#     similarities = []
#     cosine_scores = compute_cosine_similarity(query_keyword, filtered_keywords)
#     bert_scores = sentence_transformer_similarity(query_keyword, filtered_keywords)

#     for keyword in filtered_keywords:
#         wu_palmer_sim = wu_palmer_similarity(query_keyword, keyword)
#         semantic_sim = semantic_similarity(query_keyword, keyword)
#         jaccard_sim = jaccard_similarity(query_keyword, keyword)
#         levenshtein_sim = levenshtein_similarity(query_keyword, keyword)
#         cos_sim = cosine_scores[filtered_keywords.index(keyword)]
#         bert_sim = bert_scores[filtered_keywords.index(keyword)]

#         avg_score = (
#             cos_sim + wu_palmer_sim + semantic_sim +
#             jaccard_sim + levenshtein_sim + bert_sim
#         ) / 6

#         similarities.append((keyword, avg_score))

#     similarities.sort(key=lambda x: x[1], reverse=True)
#     return [kw for kw, _ in similarities[:100]]





# # Calculate similarities and get the top 100 keywords
# top_keywords = calculate_and_sort_similarities(query_keyword, filtered_keywords)

# print("Top 100 Keywords:", top_keywords)




In [112]:
# import json
# import numpy as np
# from itertools import islice
# from nltk.corpus import wordnet as wn
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from nltk.corpus import stopwords
# from difflib import SequenceMatcher
# from nltk.metrics.distance import jaccard_distance, esdit_distance
# from sentence_transformers import SentenceTransformer

# # Wu-Palmer similarity
# def wu_palmer_similarity(word1, word2):
#     syn1 = wn.synsets(word1, pos=wn.NOUN)
#     syn2 = wn.synsets(word2, pos=wn.NOUN)
#     if syn1 and syn2:
#         return max((s1.wup_similarity(s2) or 0) for s1 in syn1[:2] for s2 in syn2[:2])
#     return 0

# # Cosine similarity
# def compute_cosine_similarity(query, keywords):
#     vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=10000)
#     vectors = vectorizer.fit_transform([query] + keywords)
#     return cosine_similarity(vectors[0:1], vectors[1:]).flatten()

# # Semantic similarity (SequenceMatcher)
# def semantic_similarity(word1, word2):
#     return SequenceMatcher(None, word1, word2).ratio()

# # Jaccard similarity
# def jaccard_similarity(word1, word2):
#     set1 = set(word1.split())
#     set2 = set(word2.split())
#     return 1 - jaccard_distance(set1, set2)

# # Levenshtein similarity
# def levenshtein_similarity(word1, word2):
#     max_len = max(len(word1), len(word2))
#     if max_len == 0:
#         return 1
#     return 1 - edit_distance(word1, word2) / max_len

# # BERT-based similarity
# def sentence_transformer_similarity(query, keywords):
#     model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#     embeddings = model.encode([query] + keywords)
#     return cosine_similarity([embeddings[0]], embeddings[1:]).flatten()

# # Batch processing
# def batch_process(iterable, batch_size):
#     it = iter(iterable)
#     while batch := list(islice(it, batch_size)):
#         yield batch

# # Calculate and sort similarities
# def calculate_and_sort_similarities(query_keyword, filtered_keywords):
#     # Storage for each method's results
#     results = {
#         "Wu-Palmer Similarity": [],
#         "Semantic Similarity (SequenceMatcher)": [],
#         "Jaccard Similarity": [],
#         "Levenshtein Similarity": [],
#         "Cosine Similarity (TF-IDF)": [],
#         "BERT Similarity (SentenceTransformer)": [],
#     }

#     # Compute similarities for all methods
#     cosine_scores = compute_cosine_similarity(query_keyword, filtered_keywords)
#     bert_scores = sentence_transformer_similarity(query_keyword, filtered_keywords)

#     for keyword in filtered_keywords:
#         wu_palmer_sim = wu_palmer_similarity(query_keyword, keyword)
#         semantic_sim = semantic_similarity(query_keyword, keyword)
#         jaccard_sim = jaccard_similarity(query_keyword, keyword)
#         levenshtein_sim = levenshtein_similarity(query_keyword, keyword)
#         cos_sim = cosine_scores[filtered_keywords.index(keyword)]
#         bert_sim = bert_scores[filtered_keywords.index(keyword)]

#         # Append scores to respective method results
#         results["Wu-Palmer Similarity"].append((keyword, wu_palmer_sim))
#         results["Semantic Similarity (SequenceMatcher)"].append((keyword, semantic_sim))
#         results["Jaccard Similarity"].append((keyword, jaccard_sim))
#         results["Levenshtein Similarity"].append((keyword, levenshtein_sim))
#         results["Cosine Similarity (TF-IDF)"].append((keyword, cos_sim))
#         results["BERT Similarity (SentenceTransformer)"].append((keyword, bert_sim))

#     # Sort each method's results by score
#     for method in results:
#         results[method].sort(key=lambda x: x[1], reverse=True)

#     # Print the top 10 keywords for each method
#     print("\n--- Top Keywords by Method ---")
#     for method, keywords in results.items():
#         print(f"\n{method}:")
#         for keyword, score in keywords[:10]:  # Print top 10 for each method
#             print(f"{keyword} (Score: {score:.4f})")

#     # Calculate combined average score
#     combined_similarities = []
#     for keyword in filtered_keywords:
#         avg_score = np.mean([
#             wu_palmer_similarity(query_keyword, keyword),
#             semantic_similarity(query_keyword, keyword),
#             jaccard_similarity(query_keyword, keyword),
#             levenshtein_similarity(query_keyword, keyword),
#             cosine_scores[filtered_keywords.index(keyword)],
#             bert_scores[filtered_keywords.index(keyword)],
#         ])
#         combined_similarities.append((keyword, avg_score))

#     # Sort combined results
#     combined_similarities.sort(key=lambda x: x[1], reverse=True)
#     return [kw for kw, _ in combined_similarities[:100]]  # Top 100 by combined score

# # Compute and display results
# top_keywords = calculate_and_sort_similarities(query_keyword, filtered_keywords)
# print("\nTop 100 Keywords by Combined Score:", top_keywords)


In [113]:
# import numpy as np
# from difflib import SequenceMatcher
# from nltk.metrics.distance import edit_distance
# from sklearn.metrics.pairwise import cosine_similarity
# from sentence_transformers import SentenceTransformer

# # Semantic similarity (SequenceMatcher)
# def semantic_similarity(word1, word2):
#     return SequenceMatcher(None, word1, word2).ratio()

# # Levenshtein similarity
# def levenshtein_similarity(word1, word2):
#     max_len = max(len(word1), len(word2))
#     if max_len == 0:
#         return 1
#     return 1 - edit_distance(word1, word2) / max_len

# # BERT-based similarity
# def sentence_transformer_similarity(query, keywords):
#     model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#     embeddings = model.encode([query] + keywords)
#     return cosine_similarity([embeddings[0]], embeddings[1:]).flatten()

# # Calculate and sort similarities
# def calculate_and_sort_similarities(query_keyword, filtered_keywords):
#     # Storage for each method's results
#     results = {
#         "Semantic Similarity (SequenceMatcher)": [],
#         "Levenshtein Similarity": [],
#         "BERT Similarity (SentenceTransformer)": [],
#     }

#     # Compute BERT similarity for all keywords
#     bert_scores = sentence_transformer_similarity(query_keyword, filtered_keywords)

#     for keyword in filtered_keywords:
#         semantic_sim = semantic_similarity(query_keyword, keyword)
#         levenshtein_sim = levenshtein_similarity(query_keyword, keyword)
#         bert_sim = bert_scores[filtered_keywords.index(keyword)]

#         # Append scores to respective method results
#         results["Semantic Similarity (SequenceMatcher)"].append((keyword, semantic_sim))
#         results["Levenshtein Similarity"].append((keyword, levenshtein_sim))
#         results["BERT Similarity (SentenceTransformer)"].append((keyword, bert_sim))

#     # Sort each method's results by score
#     for method in results:
#         results[method].sort(key=lambda x: x[1], reverse=True)

#     # Print the top 10 keywords for each method
#     print("\n--- Top Keywords by Method ---")
#     for method, keywords in results.items():
#         print(f"\n{method}:")
#         for keyword, score in keywords[:10]:  # Print top 10 for each method
#             print(f"{keyword} (Score: {score:.4f})")

#     # Calculate combined average score
#     combined_similarities = []
#     for keyword in filtered_keywords:
#         avg_score = np.mean([
#             semantic_similarity(query_keyword, keyword),
#             levenshtein_similarity(query_keyword, keyword),
#             bert_scores[filtered_keywords.index(keyword)],
#         ])
#         combined_similarities.append((keyword, avg_score))

#     # Sort combined results
#     combined_similarities.sort(key=lambda x: x[1], reverse=True)
#     return [kw for kw, _ in combined_similarities[:100]]  # Top 100 by combined score



# # Compute and display results
# top_keywords = calculate_and_sort_similarities(query_keyword, filtered_keywords)
# print("\nTop 100 Keywords by Combined Score:", top_keywords)


In [114]:
# from itertools import islice
# import json
# import numpy as np
# from nltk.corpus import wordnet as wn
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from nltk.corpus import stopwords
# from difflib import SequenceMatcher

# # Wu-Palmer similarity
# def wu_palmer_similarity(word1, word2):
#     syn1 = wn.synsets(word1, pos=wn.NOUN)
#     syn2 = wn.synsets(word2, pos=wn.NOUN)
#     if syn1 and syn2:
#         return max((s1.wup_similarity(s2) or 0) for s1 in syn1[:2] for s2 in syn2[:2])
#     return 0

# # Cosine similarity
# def compute_cosine_similarity(query, keywords):
#     vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=10000)
#     vectors = vectorizer.fit_transform([query] + keywords)
#     return cosine_similarity(vectors[0:1], vectors[1:]).flatten()

# # SequenceMatcher for semantic similarity
# def semantic_similarity(word1, word2):
#     return SequenceMatcher(None, word1, word2).ratio()

# # Batch processing
# def batch_process(iterable, batch_size):
#     it = iter(iterable)
#     while batch := list(islice(it, batch_size)):
#         yield batch

# # Calculate and sort similarities
# def calculate_and_sort_similarities(query_keyword, filtered_keywords):
#     similarities = []
#     cosine_scores = compute_cosine_similarity(query_keyword, filtered_keywords)

#     for batch in batch_process(filtered_keywords, batch_size=100):  # Process 100 keywords at a time
#         for keyword in batch:
#             wu_palmer_sim = wu_palmer_similarity(query_keyword, keyword)
#             semantic_sim = semantic_similarity(query_keyword, keyword)
#             cos_sim = cosine_scores[filtered_keywords.index(keyword)]

#             avg_score = (cos_sim + wu_palmer_sim + semantic_sim) / 3
#             similarities.append((keyword, avg_score))

#     similarities.sort(key=lambda x: x[1], reverse=True)
#     return [kw for kw, _ in similarities[:100]]


# file_path = "filtered_keywords.json"

# # with open(file_path, 'r') as f:
# #     data = json.load(f)

# top_keywords = calculate_and_sort_similarities(query_keyword, filtered_keywords)

# print("Top 100 Keywords:", top_keywords)


In [115]:
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# from sentence_transformers import SentenceTransformer

# # BERT-based similarity
# def sentence_transformer_similarity(query, keywords):
#     model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#     embeddings = model.encode([query] + keywords)
#     return cosine_similarity([embeddings[0]], embeddings[1:]).flatten()

# # Calculate and sort similarities
# def calculate_and_sort_similarities(query_keyword, filtered_keywords):
#     # Compute BERT similarity for all keywords
#     bert_scores = sentence_transformer_similarity(query_keyword, filtered_keywords)

#     # Store results
#     results = {"BERT Similarity (SentenceTransformer)": []}

#     for keyword, bert_sim in zip(filtered_keywords, bert_scores):
#         results["BERT Similarity (SentenceTransformer)"].append((keyword, bert_sim))

#     # Sort results by score
#     results["BERT Similarity (SentenceTransformer)"].sort(key=lambda x: x[1], reverse=True)

#     # Print the top 10 keywords
#     print("\n--- Top Keywords by BERT Similarity ---")
#     for keyword, score in results["BERT Similarity (SentenceTransformer)"][:10]:
#         print(f"{keyword} (Score: {score:.4f})")

#     # Return top 100 keywords by BERT similarity
#     return [kw for kw, _ in results["BERT Similarity (SentenceTransformer)"][:100]]



# # Compute and display results
# top_keywords = calculate_and_sort_similarities(query_keyword, filtered_keywords)
# print("\nTop 100 Keywords by BERT Similarity:", top_keywords)
